<h1><b><center>How to use this file with PyRat?</center></b></h1>

Google Colab provides an efficient environment for writing codes collaboratively with your group. For us, teachers, it allows to come and see your advancement from time to time, and help you solve some bugs if needed.

The PyRat software is a complex environment that takes as an input an AI file (as this file). It requires some resources as well as a few Python libraries, so we have installed it on a virtual machine for you.

PyRat is a local program, and Google Colab is a distant tool. Therefore, we need to indicate the PyRat software where to get your code! In order to submit your program to PyRat, you should follow the following steps:

1.   In this notebook, click on *Share* (top right corner of the navigator). Then, change sharing method to *Anyone with the link*, and copy the sharing link;
2.   On the machine where the PyRat software is installed, start a terminal and navigate to your PyRat directory;
3.   Run the command `python ./pyrat.py --rat "<link>" <options>`, where `<link>` is the share link copied in step 1. (put it between quotes), and `<options>` are other PyRat options you may need.

<h1><b><center>Pre-defined constants</center></b></h1>

A PyRat program should -- at each turn -- decide in which direction to move. This is done in the `turn` function later in this document, which should return one of the following constants:

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'

<h1><b><center>Your coding area</center></b></h1>

Please put your functions, imports, constants, etc. between this text and the PyRat functions below. You can add as many code cells as you want, we just ask that you keep things organized (one function per cell, commented, etc.), so that it is easier for the teachers to help you debug your code!

In [ ]:
#Importation module
import random as rd
import heapq
import copy as c

#python pyrat.py --rat "" --python "https://colab.research.google.com/drive/1tSogjcQWO2cdv_WcRPsMTqhfp5cWVnFE?usp=sharing" --tests 100 --nodrawing -x 10 -y 10 -p 6 -md 0.0 --synchronous
#python pyrat.py --rat "https://colab.research.google.com/drive/1tSogjcQWO2cdv_WcRPsMTqhfp5cWVnFE?usp=sharing" -md 0.0 -p 1 -x 5 -y 5 --random_seed 1
#python pyrat.py --rat "https://colab.research.google.com/drive/1FLItxuxxs9-53U7gVDPxSxR3OlYIh1FS?usp=sharing" -p 10 -x 15 -y 15 -d 0.5 --random_seed 1

#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing" -p 9 -x 15 -y 15 --random_seed 1
#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing" --tests 1 --nodrawing -x 50 -y 50 -p 34
#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing" --python "https://colab.research.google.com/drive/1tSogjcQWO2cdv_WcRPsMTqhfp5cWVnFE?usp=sharing" --tests 100 --nodrawing -x 10 -y 10 -p 6 -md 0.0 --synchronous

#Greedy 4-4 VS Greedy 1-1
#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing"
#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing" --python "https://colab.research.google.com/drive/1KcUmZoO1xkQ477U0p76u628dVORcT25E?usp=sharing"
#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing" --python "https://colab.research.google.com/drive/1KcUmZoO1xkQ477U0p76u628dVORcT25E?usp=sharing" --tests 50 --nodrawing -x 10 -y 10 -p 6

#Greedy 4-4 VS DFS
#python pyrat.py --rat "https://colab.research.google.com/drive/15eVr2OvcvjBGmvbJF-BwJxVDCFLEssWX?usp=sharing" --python "https://colab.research.google.com/drive/1tSogjcQWO2cdv_WcRPsMTqhfp5cWVnFE?usp=sharing"

In [ ]:
#Variable global
meta_graph = dict()
routes = list()
best_path = list()
cheeses_with_score = dict()
best_length = 1e40
index_route=-1
index_cheese = 0
remaining = []
fluctuant_remaining = []
actualize_greedy = False
centerphase = True
center_route = []
index = -1

#Indice du nombre de voisin le plus proche à étudier pour le greedy
n = 4


#Stratégie : Greedy de niveau 4 + Privilégier les régions où la densité de fromage est plus grande + Aller dans le centre

In [ ]:
def move_from_locations(source_location, target_location):

  difference = (target_location[0] - source_location[0], target_location[1] - source_location[1])
  if difference == (0, -1):
    return MOVE_DOWN
  elif difference == (0, 1):
    return MOVE_UP
  elif difference == (1, 0):
    return MOVE_RIGHT
  elif difference == (-1, 0):
    return MOVE_LEFT
  else :
    raise Exception("Impossible move")

In [ ]:
def dijkstra(player_location, maze_map):
  '''Renvoie la routing table de la maze_map à partir de player_location'''
  #initialize
  priority_queue = list()
  visited_locations=list()
  heapq.heappush(priority_queue, (0, (player_location, None)))
  routing_table = dict()
  #algorithm loop
  while priority_queue != list() and len(routing_table)!=len(maze_map):
    distance, vertices = heapq.heappop(priority_queue)
    visited_locations.append(vertices[0])
    routing_table[vertices[0]] = vertices[1]
    #Parcours les voisins de v[0] = current_location
    for neighbor in maze_map[vertices[0]].keys():
      distance_of_neighbor = distance + maze_map[vertices[0]][neighbor]
      if neighbor not in visited_locations:
        heapq.heappush(priority_queue, (distance_of_neighbor, (neighbor, vertices[0])))

  return routing_table

In [ ]:
def finds_route(maze_map, routing_table, source_location, target_location):
  '''Renvoie la distance et la route de source_location à targe_location dans une liste de positions'''
  route = list()
  location = target_location
  distance  = 0
  #On parcours le routing_table à l'envers à partir du fromage jusqu'au source_location
  while location != source_location:
    route.insert(0, location)
    ##print(source_location,location,routing_table[location])
    distance += maze_map[routing_table[location]][location]
    #Location devient son parent
    location = routing_table[location]

  route.insert(0, source_location)

  return route, distance

In [ ]:
def build_meta_graph(maze_map, center_location, pieces_of_cheese):
  '''Return the meta-graph and all necessary routing tables'''
  #Meta Graph est un dictionnaire de dictionnaire de tuple (distance, route_associé)
  #On cherche à créer un graph complèt ne contenant que des fromages et la position initiale

  global meta_graph
  locations = c.deepcopy(pieces_of_cheese)
  locations.insert(0, center_location)

  for k in range(len(locations)):
    #Calcul le routing_table à partir du vertices location
    routing_table=dijkstra(locations[k],maze_map)
    meta_graph[locations[k]]=dict()

    for i in range(len(locations)):
      if i != k:
        #Route_ki est la route réel à partir entre location[k] et location[i] et distance est sa distance totale
        route_ki, distance = finds_route(maze_map, routing_table, locations[k], locations[i])

        meta_graph[locations[k]][locations[i]] = (distance, route_ki)
      else:
        meta_graph[locations[k]][locations[i]] = (0,[])


In [ ]:
def center(player_location, maze_map, center_location, pieces_of_cheese, k):
  '''Envoie le player_location dans le centre en debut de partie en prenant les fromages les plus faciles'''

  #Determine la route à suivre la plus courte suivant l'algorithme de Dijkstra
  routing_table = dijkstra(player_location, maze_map)
  route, distance = finds_route(maze_map, routing_table, player_location, center_location)

  #On crée une liste que l'on modifié pour ne pas manger deux fois le même fromage dans la phase "Aller au centre"
  started_cheese = c.deepcopy(pieces_of_cheese)

  #Cherche les fromages les plus proches à 3 cases maximum avec un distance max de k de la trajectoire de la route
  #On parcours chaque élément de la route vers le centre
  for location in route:
    index = route.index(location)

    #Parcours les voisins de chacun des élements de la route
    for first_neighbor in maze_map[location].keys():
      dist1 = maze_map[location][first_neighbor]
      if first_neighbor in started_cheese and dist1 <= k and first_neighbor not in route:

        #On insert la position du premier_voisin dans la route final si celui-ci est un fromage
        route = route[:index+1] + [first_neighbor] + route[index:]
        started_cheese.remove(first_neighbor)

      #Parcours des voisins des premiers voisins
      for second_neighbor in maze_map[first_neighbor].keys():
        dist2 = dist1 + maze_map[first_neighbor][second_neighbor]
        if second_neighbor in started_cheese and dist2 <= k and second_neighbor not in route and first_neighbor not in route:

          #De même, on insert le chemin à parcours si les seconds voisins est un fromage
          route = route[:index+1] + [first_neighbor,second_neighbor,first_neighbor] + route[index:]
          started_cheese.remove(second_neighbor)

        #Parcours des voisins des seconds voisins
        for third_neighbor in maze_map[second_neighbor].keys():
          dist3 = dist2 + maze_map[second_neighbor][third_neighbor]
          if second_neighbor in started_cheese and dist3 <= k and third_neighbor not in route and second_neighbor not in route and first_neighbor not in route:

            #Idem pour les troisièmes voisins
            route = route[:index+1] + [first_neighbor,second_neighbor, third_neighbor, second_neighbor,first_neighbor] + route[index:]
            started_cheese.remove(third_neighbor)


  return(route)

In [ ]:
def density(cheese):
  '''Renvoie la densité associé au fromage cheese'''
  global meta_graph
  global remaining

  #Calculer la densité dans le préprocessing

  somme_distances = 0
  densité = 0

  #Calcul la densité de fromage aux alentours de cheese
  for other_cheese in remaining:
    somme_distances += meta_graph[cheese][other_cheese][0]

  #Le choix du facteur 10 et la définition de notre densité sont arbitraire
  #On évite de diviser par 0 pour le dernier fromage
  densité = 100*len(remaining)/(float(somme_distances) + 0.1)
  return densité

In [ ]:
def closed_neighbors(location, fluctuant_remaining, n):
  '''Return the n-th closest and available neighbors in the meta_graph from location'''
  global meta_graph

  #Parcours le meta_graph pour crée une minheap des voisins du fromage location suivant la densité des fromages voisins
  priority_queue = []

  for neighbor in meta_graph[location].keys():
    if neighbor in fluctuant_remaining:
      heapq.heappush(priority_queue, (meta_graph[location][neighbor][0]*density(neighbor), neighbor))

  #Sélectionne les n fromages les plus proches de location
  closest = []

  for i in range(min(len(priority_queue),n)):
    closest.append(heapq.heappop(priority_queue)[1])

  return(closest)


In [ ]:
def greedy5(player_location, n):
  '''Renvoie la liste des voisins 5ième plus courts de player_location dans le metagraph  à étudier pour déterminer le chemin le plus court'''

#Programme pour Greedy de niveau 4 mais en évaluant que le n meilleurs chemins à chaque étape
#Arbre de profondeur 4 et de largeur n

  global best_length
  global best_path
  global meta_graph
  global remaining

  #fluctuant_remaining est un remaining actualisé à chaque étaque du greedy pour éviter que le voisin d'un voisin revienne sur ce voisin
  fluctuant_remaining = c.deepcopy(remaining)

  for n1 in closed_neighbors(player_location, fluctuant_remaining, n):

    #Le score est égale au produit de la distance avec la densité
    score_so_far = meta_graph[player_location][n1][0]*density(n1)
    #Pour que n2 ne considère pas n1
    fluctuant_remaining.remove(n1)

    #Adapte le code si n1 n'a pas de voisin
    closed_n1 = closed_neighbors(n1, fluctuant_remaining, n)

    if closed_n1 != []:
      #Garde en mémoire le meilleur score pour revenir à score_so_far à chaque futur tour de boucle pour les voisins de n1
      score_memory1 = c.deepcopy(score_so_far)

      for n2 in closed_n1:
        #Même principe que pour le premier voisin
        score_so_far = c.deepcopy(score_memory1)
        score_so_far += meta_graph[n1][n2][0]*density(n2)
        fluctuant_remaining.remove(n2)
        closed_n2 = closed_neighbors(n2, fluctuant_remaining, n)

        if closed_n2 != []:
          score_memory2 = c.deepcopy(score_so_far)

          for n3 in closed_n2:
            #Même principe que pour le premier voisin
            score_so_far = c.deepcopy(score_memory2)
            score_so_far += meta_graph[n2][n3][0]*density(n3)

            fluctuant_remaining.remove(n3)
            closed_n3 = closed_neighbors(n3, fluctuant_remaining, n)

            if closed_n3 != []:
              score_memory3 = score_so_far

              for n4 in closed_n3:
                #Même principe que pour le premier voisin
                score_so_far = c.deepcopy(score_memory3)
                score_so_far += meta_graph[n3][n4][0]*density(n4)

                #On actualise le meilleur chemin
                if score_so_far < best_length:
                  best_length = score_so_far
                  best_path = [player_location, n1, n2, n3, n4]

            #On actualise meilleur chemin si closed_n3 == []
            elif score_so_far < best_length:
              best_length = score_so_far
              best_path = [player_location, n1, n2, n3]
            fluctuant_remaining.append(n3)

        #Idem si closed_n2 == []
        elif score_so_far < best_length:
          best_length = score_so_far
          best_path = [player_location, n1, n2]
        fluctuant_remaining.append(n2)

    #Idem si closed_n1 == []
    elif score_so_far < best_length:
      best_length = score_so_far
      best_path = [player_location, n1]
    fluctuant_remaining.append(n1)
    print(best_length)
  print(best_path)

In [ ]:
def actualize(player_location):
  '''On actualise le best_path et le best_length tous les cinqs fromages mangés'''
  global best_length
  global routes
  global actualize_greedy
  global remaining
  global n

  #On met actualize_greedy en False pour éviter qu'il ne soit tous les tours
  actualize_greedy = False
  best_length = 1e40
  greedy5(player_location, n)
  print("Actualisation")

  meta_graph_route_to_route()

In [ ]:
def meta_graph_route_to_route() :
  # Return the sequence of locations in the maze to perform a route in the meta-graph
  global routes
  global best_path
  global meta_graph

  #On concatène les routes à partir du best_path
  for i in range(len(best_path)-1):
    routes += meta_graph[best_path[i]][best_path[i+1]][1][1:]

<h1><b><center>PyRat functions</center></b></h1>

The `preprocessing` function is called at the very start of a game. It is attributed a longer time compared to `turn`, which allows you to perform intensive computations. If you store the results of these computations into **global** variables, you will be able to reuse them in the `turn` function.

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze where the game takes place. This structure associates each cell with the dictionry of its neighbors. In that dictionary of neighbors, keys are cell coordinates, and associated values the number of moves required to reach that neighbor. As an example, `list(maze_map[(0, 0)].keys())` returns the list of accessible cells from `(0, 0)`. Then, if for example `(0, 1)` belongs to that list, one can access the number of moves to go from `(0, 0)` to `(0, 1)` by the code `maze_map[(0, 0)][0, 1)]`.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The initial location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The initial location of your opponent's character in the maze.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The initial location of all pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take for preprocessing before the game starts checking for moves.

*Output:*
*   This function does not output anything.

In [ ]:
def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :

    global meta_graph
    global routes
    global remaining
    global center_route
    global actualize_greedy
    global centerphase
    global meta_graph

    #On copie la liste des fromages que l'on adaptera suivant les fromages mangés
    remaining = c.deepcopy(pieces_of_cheese)


    #Execute l'algorithme pour aller vers le centre si les tailles de la maze_map sont suffisamments grandes (Choix arbitraire)
    #En effet, nous avons remarqué que la stratégie d'aller au centre n'est pas très efficace pour de petites maze_map
    if maze_width*maze_height >= 400:
      print("Allons vers le centre")
      center_location = (int(maze_width/2),int(maze_height/2))
      center_route = center(player_location, maze_map, center_location, pieces_of_cheese, 8)
      routes = c.deepcopy(center_route)

      #On construit le meta_graph à partir du centre: Graph complet constitué des fromages et de position initiale
      build_meta_graph(maze_map, center_location, pieces_of_cheese)

    #Si la maze_map n'est pas assez grande, on lance notre greedy 4-4
    else:
      print("Pas besoin d'aller au centre, lancement Greedy!")
      routes.insert(0, player_location)
      build_meta_graph(maze_map, player_location, pieces_of_cheese)
      print(meta_graph)
      actualize(player_location)
      centerphase = False




The `turn` function is called each time the game is waiting
for the player to make a decision (*i.e.*, to return a move among those defined above).

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze. It is the same as in the `preprocessing` function, just given here again for convenience.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The current location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The current location of your opponent's character in the maze.
*   `player_score` -- **float** -- Your current score.
*   `opponent_score` -- **float** -- The opponent's current score.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The location of remaining pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take to return a move to apply before another time starts automatically.

*Output:*
*   A chosen move among `MOVE_UP`, `MOVE_DOWN`, `MOVE_LEFT` or `MOVE_RIGHT`.

In [ ]:
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
  global index_route
  global remaining
  global index_cheese
  global actualize_greedy
  global center_route
  global centerphase
  global routes

  index_route += 1

  #Code pour le début de partie
  if center_route != [] :
    #Actualise le remaining si un fromage est mangé
    if player_location in remaining:
      remaining.remove(player_location)
    center_route.remove(player_location)

  #L'algorithme de Greedy commence quand on arrive au centre
  if player_location == (int(maze_width/2),int(maze_height/2)) and centerphase:
    print("Arrivé au centre")
    actualize_greedy = True
    centerphase = False

  #Actualise les fromages restants (Periode après centre), l'algorithme de greedy s'actualise tous les quatres fromages mangés.
  if player_location in remaining and not centerphase:
    index_cheese += 1
    if index_cheese % 4 == 0:
      actualize_greedy = True
    remaining.remove(player_location)

  #Actualise les fromages restants après que l'adversaire en mange un
  if opponent_location in remaining:
    remaining.remove(opponent_location)

    if opponent_location in best_path:
      index_cheese += 1


  #Actualise le greedy quand il le faut
  if actualize_greedy:
    actualize(player_location)

  #Reactualiser le greedy en supprimant la route à ne pas parcourir - A faire pour plus tard
  return move_from_locations(routes[index_route], routes[index_route + 1])